# Creating Binary Variables

### Import the Packages

In [1]:
import pandas as pd
import numpy as np
import os 

### Load the Dataset

In [2]:
filename = os.path.join(os.getcwd(), "data", "adult.data.partial")
df = pd.read_csv(filename, header=0)

### Glance at the Data

In [3]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label
0,36,State-gov,112074,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Non-Female,0,0,45,United-States,<=50K
1,35,Private,32528,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Non-Female,0,0,45,United-States,<=50K
2,21,Private,270043,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,16,United-States,<=50K
3,45,Private,168837,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,24,Canada,>50K
4,39,Private,297449,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Non-Female,0,0,40,United-States,>50K


### Display Summary Statistics by Column

In [4]:
df.describe(include='all')

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label
count,7000.000000,6625,7.000000e+03,7000,7000.000000,7000,6625,7000,7000,7000,7000.000000,7000.000000,7000.000000,6862,7000
unique,NaN,7,NaN,16,NaN,7,14,6,5,2,NaN,NaN,NaN,40,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Non-Female,NaN,NaN,NaN,United-States,<=50K
freq,NaN,4879,NaN,2263,NaN,3277,911,2878,5990,4731,NaN,NaN,NaN,6233,5319
mean,38.596714,NaN,1.924335e+05,NaN,10.049857,NaN,NaN,NaN,NaN,NaN,1079.000429,84.970286,40.107143,NaN,NaN
std,13.745594,NaN,1.063365e+05,NaN,2.580982,NaN,NaN,NaN,NaN,NaN,7011.160679,400.142351,12.323946,NaN,NaN
min,17.000000,NaN,1.882700e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.202478e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.821170e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,47.000000,NaN,2.402370e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


## Group Columns into Binary Values Using `np.where()` 


One of the typical tasks in data cleaning and feature engineering is to convert a feature or a label that has multiple categorical values into one that has a binary value. This means that instead of having multiple potential values, a feature or a label will have just two potential values.

For example, let's say we have "type of animal" as a feature and a list of 11 potential values for animals:

cats, dogs, sharks, elephants, iguanas, pigeons, humans, dolphins, mice, goldfish, hummingbirds

We can group these animals into **two** categories- `mammal` and `not-mammal` - and therefore, the "type of animal" feature will have two potential values (rather than 11 potential values).

cats, dogs, elephants, humans, dolphins, mice -> **mammal** 

iguanas, pigeons, goldfish, sharks, hummingbirds -> **not-mammal**


In this exercise, we will convert one feature column in our dataset (the `workclass` column) to contain binary values.


In the cell below, inspect the current the `workclass` column in the DataFrame `df`. Notice the different feature values.

In [5]:
df['workclass'].unique()

array(['State-gov', 'Private', nan, 'Self-emp-not-inc', 'Local-gov',
       'Self-emp-inc', 'Federal-gov', 'Without-pay'], dtype=object)

Inspect the first 30 rows of DataFrame `df` and focus on the `workclass` column to see some of the above values.

In [8]:
df.head(30)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label
0,36,State-gov,112074,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Non-Female,0,0,45,United-States,<=50K
1,35,Private,32528,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Non-Female,0,0,45,United-States,<=50K
2,21,Private,270043,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,16,United-States,<=50K
3,45,Private,168837,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,24,Canada,>50K
4,39,Private,297449,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Non-Female,0,0,40,United-States,>50K
5,27,Private,233421,Some-college,10,Never-married,Adm-clerical,Own-child,White,Non-Female,0,0,20,United-States,<=50K
6,40,Private,220460,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Non-Female,0,0,40,Canada,<=50K
7,71,Private,163385,Some-college,10,Widowed,Sales,Not-in-family,White,Non-Female,0,0,35,United-States,>50K
8,20,NaN,193416,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,40,United-States,<=50K
9,41,Private,116391,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Non-Female,0,0,40,United-States,<=50K


Our goal is to consolidate all of the different types of employment into two groups: **self-employed** and **not self-employed**, and change the values in the `workclass` column accordingly.


### Step 1: Create Group 1: `Not-self-emp`

Notice that the `workclass` column contains the values `Self-emp-inc`and `Self-emp-not-inc` to indicate that an individiual is self-employed (these correspond to incorporated and the unincorporated self-employment).


Note that the `workclass` column contains a number of different values for individuals who are *not* self employed ('State-gov', 'Private', 'Local-gov', 'Federal-gov', 'Without-pay').

As a first step, we can group all of the "not self employed" values into one category - a **not self employed** category. We can change all of the `workclass` columns that contain the values 'State-gov', 'Private', 'Local-gov', 'Federal-gov', 'Without-pay' to the value `'Not-self-emp'`.

Let's try this using the `np.where()` function. First, read the documentation for the `np.where()` function. Then examine and run the code below and try to understand how it works.

For more information about `np.where()`, consult the online [documentation](https://numpy.org/doc/stable/reference/generated/numpy.where.html).


In [12]:
np.where?

In [10]:
# Since there are only two values for self-employment, we can simplify our code by writing 
# NOT self employed 

# get all examples (rows) in which the workclass feature (column) is not self-employed
# Note: the code below uses the Pandas logical operator ~ for NOT
columns_not_self_employed = ~(df['workclass'] == 'Self-emp-not-inc') & ~(df['workclass'] == 'Self-emp-inc')

#leave nan (null) in the dataset for now. Get all examples (rows) in which the workclass feature (column) is not null
columns_not_null = ~(df['workclass'].isnull())  

# create condition
condition = columns_not_self_employed & columns_not_null

# Use np.where() to change all of the workclass values that fulfill the specified condition to Not-self-emp
df['workclass'] = np.where(condition, 'Not-self-emp', df['workclass'])

# Inspect the data to see the changed values
df.head(30)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label
0,36,Not-self-emp,112074,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Non-Female,0,0,45,United-States,<=50K
1,35,Not-self-emp,32528,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Non-Female,0,0,45,United-States,<=50K
2,21,Not-self-emp,270043,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,16,United-States,<=50K
3,45,Not-self-emp,168837,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,24,Canada,>50K
4,39,Not-self-emp,297449,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Non-Female,0,0,40,United-States,>50K
5,27,Not-self-emp,233421,Some-college,10,Never-married,Adm-clerical,Own-child,White,Non-Female,0,0,20,United-States,<=50K
6,40,Not-self-emp,220460,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Non-Female,0,0,40,Canada,<=50K
7,71,Not-self-emp,163385,Some-college,10,Widowed,Sales,Not-in-family,White,Non-Female,0,0,35,United-States,>50K
8,20,NaN,193416,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,40,United-States,<=50K
9,41,Not-self-emp,116391,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Non-Female,0,0,40,United-States,<=50K


Run the cell below to see the new  feature values for the `workclass` feature.

In [11]:
df['workclass'].unique()

array(['Not-self-emp', nan, 'Self-emp-not-inc', 'Self-emp-inc'],
      dtype=object)

Notice that we did not lose the `nan` values. Think about what would have happened had we not excluded 'NaN' values in our 'condition'.

### Step 2: Create Group 2: `Self-emp`

We have successfully created a `Not-self-emp` group and changed all pertinent values in column `workclass` to the value `Not-self-emp`. 

However, recall that our goal was to consolidate all of the different types of employment into two groups: **self-employed** and **not self-employed**

Notice that we still have two values for self employment: `Self-emp-not-inc`, `Self-emp-inc`.

We do not care to distinguish between the incorporated and the unincorporated self-employed examples, so we will next consolidate and change all `Self-emp-not-inc`and `Self-emp-inc` values in the `workclass` column to the value `Self-emp`.

In the two code cells below, you will use `np.where()` to combine `Self-emp-not-inc` and `Self-emp-inc` into a single class called `Self-emp`. Note: Follow the two steps as outlined per cell in order to properly grade your work.

#### Part 1:
Create the condition and name it `condition`. `condition` will contain a compound statement joined together by the Boolean operator <b>or</b> (`|`). It will find the people who are self employed, i.e. it will look for both the value `Self-emp-not-inc` and the value `Self-emp-inc` in the `workclass` column. 


### Graded Cells

The two cells below will be graded. Remove the lines "raise NotImplementedError()" before writing your code.

In [16]:
# YOUR CODE HERE
condition = (df['workclass'] == 'Self-emp-not-inc') | (df['workclass'] == 'Self-emp-inc')

### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [17]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testCondition

try:
    p, err = testCondition(df, condition)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    


Correct!


#### Part 2:

In the code cell below, use `np.where()` to combine `Self-emp-not-inc` and `Self-emp-inc` into a single class called `Self-emp`. Use the condition you created in Step 1. Follow the code pattern you have seen previously when implementing `np.where()`.


In [18]:
# YOUR CODE HERE
df['workclass'] = np.where(condition, 'Self-emp', df['workclass'])

### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [19]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testWorkclass

try:
    p, err = testWorkclass(df)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    


Correct!


In [20]:
# Check your results 
df['workclass'].unique()

array(['Not-self-emp', nan, 'Self-emp'], dtype=object)

The cell below transforms the labels into a binary 'True' and 'False' variable, where 'True' is the label assigned to income level >50K. Run the code and inspect the results.


In [21]:
condition1 = (df['label'] == '>50K')
df['label'] = np.where(condition1, 'True', df['label'])

condition2 = (df['label'] == '<=50K')
df['label'] = np.where(condition2, 'False', df['label'])

df.head(30)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label
0,36,Not-self-emp,112074,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Non-Female,0,0,45,United-States,False
1,35,Not-self-emp,32528,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Non-Female,0,0,45,United-States,False
2,21,Not-self-emp,270043,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,16,United-States,False
3,45,Not-self-emp,168837,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,24,Canada,True
4,39,Not-self-emp,297449,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Non-Female,0,0,40,United-States,True
5,27,Not-self-emp,233421,Some-college,10,Never-married,Adm-clerical,Own-child,White,Non-Female,0,0,20,United-States,False
6,40,Not-self-emp,220460,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Non-Female,0,0,40,Canada,False
7,71,Not-self-emp,163385,Some-college,10,Widowed,Sales,Not-in-family,White,Non-Female,0,0,35,United-States,True
8,20,NaN,193416,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,40,United-States,False
9,41,Not-self-emp,116391,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Non-Female,0,0,40,United-States,False


## Deep Dive: Categorical Variables

Let us discuss categorical variables in a little bit more detail.<br>

A categorical data variable is often more than just a variable who's value is one of two or more classes, or categories.<br>
Sometimes the *order* of categories is meaningful, and contains some information that may be useful for analysis.<br>
For example, we may have a dataset that contains a variable 'ice_cream_flavor' that takes five possible values: 'vanilla', 'strawberry', 'pistachio', 'chocolate', and 'mango'. In this example, there is not a natural order to the five categories. <br>
Now consider a variable called 'portion_size' that takes on one of the three possible values: 'standard', 'double', and 'super'. In this case, the three categories are related to one another by an *ordering*. We would like for any future model we fit to this data to recognize and make use of the fact that $$standard < double < super.$$
<br>

Try to think of a variable in our dataset that should be an ordered categorical variable.

One such variable would be 'education'. Let's examine the list of possible values:

In [22]:
df['education'].unique()

array(['Doctorate', 'HS-grad', 'Some-college', 'Bachelors', '9th',
       'Masters', 'Assoc-voc', '10th', 'Prof-school', '7th-8th',
       'Assoc-acdm', '11th', '5th-6th', '1st-4th', '12th', 'Preschool'],
      dtype=object)

There is clearly an underlying order to these categories!<br>
In general, you would have to consult the data manual to establish what the correct order is.<br>
For this data, the true ordering would be:<br>
    
$$Preschool < 1st-4th < 5th-6th < 7th-8th < 9th < 10th < 11th < 12th < HS-grad < Prof-school < Assoc-acdm < Assoc-voc < Some-college < Bachelors < Masters < Doctorate.$$
<br>
Is this variable currently ordered?

In [23]:
df['education'].dtype

dtype('O')

No! This variable is of type 'object'. Any future analysis in python won't recognize the underlying order of categories. 
<br> We need to transform this variable into an ordered categorical variable. Here's how.<br>

In [24]:
# First, create a correctly ordered list of category names:
edu = ['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '1th', '12th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Some-college', 'Bachelors', 'Masters', 'Doctorate']
# Then, use the pd.Categorical method to reassign the values of this column as a new type with awareness of the order:
df['education'] = pd.Categorical(df['education'], ordered=True, categories=edu)


In [25]:
df['education'].dtype

CategoricalDtype(categories=['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th',
                  '1th', '12th', 'HS-grad', 'Prof-school', 'Assoc-acdm',
                  'Assoc-voc', 'Some-college', 'Bachelors', 'Masters',
                  'Doctorate'],
                 ordered=True)

Perfect! Now we can be sure that the order is recognized by Python. <br>
(Note: This will come in handy when we create plots with categories on one axis.)

### Convert Categorical Variables into "Dummy" Variables

There are many different ways we can transform our categorical data into numerical data to prepare for the model training phase. Pandas has one function that helps us transform categorical values into binary ones. It is the `pd.get_dummies()` function. Often we refer to these binary values as "dummy" values, or variables. Read up on the Pandas `pd.get_dummies()` function. You can consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).
Run the cell below to observe the results of creating dummies for all variables in our data.

In [26]:
df_binary = pd.get_dummies(df)
df_binary.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Not-self-emp,workclass_Self-emp,education_Preschool,education_1st-4th,...,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,label_False,label_True
0,36,112074,16,0,0,45,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,35,32528,9,0,0,45,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,21,270043,10,0,0,16,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,45,168837,10,0,0,24,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,39,297449,13,0,0,40,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
